# Train a Simple Combinatorial Complex Convolutional Neural Network for Mesh Classification.

We create and train a mesh classification high order attentional neural network operating over combinatorial complexes. 

## The Neural Network:

The neural network is composed of a sequence of identical convolutional layers for a dimension two combinatorial complex, a final fully connected layer embedding the features into a common space, and a final transformation to a vector with probabilities for each class. Each attention layer is composed of two levels. In both levels, messages computed for the cells of identical dimension are aggregated using a sum operation. All the messages are computed using the attention mechanisms for squared and non-squared neighborhoods presented in [Definitions 31, 32, and 33, Hajij et. al : Topological Deep Learning: Going Beyond Graph Data (2023)](https://arxiv.org/pdf/2206.00606.pdf).


## The Task:

We train this model to perform entire mesh classification on [`SHREC 2016` from the ShapeNet Dataset](http://shapenet.cs.stanford.edu/shrec16/). This dataset contains 480 3D mesh samples belonging to 30 distinct classes and represented as simplicial complexes.

Each mesh contains a set of vertices, edges, and faces. Each of the latter entities have a set of features associated to them:

- Node features $v \in \mathbb{R}^6$ defined as the direct sum of the following features:
    - Position $p_v \in \mathbb{R}^3$ coordinates.
    - Normal $n_v \in \mathbb{R}^3$ coordinates.
- Edge features $e \in \mathbb{R}^{10}$ defined as the direct sum of the following features:
    - Dihedral angle $\phi \in \mathbb{R}$.
    - Edge span $l \in \mathbb{R}$.
    - 2 edge angle in the triangle that $\theta_e \in \mathbb{R}^2$.
    - 6 edge ratios $r \in \mathbb{R}^6$.
- Face features
    - Face area $a \in \mathbb{R}$.
    - Face normal $n_f \in \mathbb{R}^3$.
    - 3 face angles $\theta_f \in \mathbb{R}^3$.

We lift the simplicial complexes representing each mesh to a topologically equivalent combinatorial complex representation.

The task is to predict the class that a certain mesh belongs to, given its combinatorial complex representation. For this purpose we implement the Higher Order Attention Model for Mesh Classification first introduced in [Hajij et. al : Topological Deep Learning: Going Beyond Graph Data (2023)](https://arxiv.org/pdf/2206.00606.pdf).

# Set-up


In [1]:
import torch
import numpy as np
import toponetx
# from toponetx.datasets.mesh import shrec_16
import toponetx.datasets as datasets

# from toponetx.datasets import mesh
# import toponetx.datasets.mesh as mesh
# from toponetx.datasets.mesh import shrec_16
from torch.utils.data import Dataset, DataLoader
# from topomodelx.nn.combinatorial.hmc_layer import HMCLayer
# from topomodelx.nn.combinatorial.hmc import HMC
from typing import Tuple, List
from topomodelx.base import Conv

# Pre-processing

## Import data ##

We first create a class for the SHREC 2016 dataset. This class will be used to load the data and create the necessary neighborhood matrices for each combinatorial complex in the dataset.

In [2]:
class SHRECDataset(Dataset):
    """Class for the SHREC 2016 dataset.

    Parameters
    ----------
    data : npz file
        npz file containing the SHREC 2016 data.
    """

    def __init__(self, data):
        self.complexes = [cc.to_combinatorial_complex() for cc in data["complexes"]]
        self.x_0 = data["node_feat"]
        self.x_1 = data["edge_feat"]
        self.x_2 = data["face_feat"]
        self.y = data["label"]
        self.a0, self.a1, self.coa2, self.b1, self.b2 = self._get_neighborhood_matrix()

    def _get_neighborhood_matrix(self):
        """Neighborhood matrices for each combinatorial complex in the dataset.

        Following the Higher Order Attention Model for Mesh Classification message passing scheme, this method computes the necessary neighborhood matrices
        for each combinatorial complex in the dataset. This method computes:

        - Adjacency matrices for each 0-cell in the dataset.
        - Adjacency matrices for each 1-cell in the dataset.
        - Coadjacency matrices for each 2-cell in the dataset.
        - Incidence matrices from 1-cells to 0-cells for each 1-cell in the dataset.
        - Incidence matrices from 2-cells to 1-cells for each 2-cell in the dataset.

        Returns
        -------
        a0 : list of torch.sparse.FloatTensor
            Adjacency matrices for each 0-cell in the dataset.
        a1 : list of torch.sparse.FloatTensor
            Adjacency matrices for each 1-cell in the dataset.
        coa2 : list of torch.sparse.FloatTensor
            Coadjacency matrices for each 2-cell in the dataset.
        b1 : list of torch.sparse.FloatTensor
            Incidence matrices from 1-cells to 0-cells for each 1-cell in the dataset.
        b2 : list of torch.sparse.FloatTensor
            Incidence matrices from 2-cells to 1-cells for each 2-cell in the dataset.
        """

        a0 = []
        a1 = []
        coa2 = []
        b1 = []
        b2 = []

        for cc in self.complexes:
            a0.append(torch.from_numpy(cc.adjacency_matrix(0, 1).todense()).to_sparse())
            a1.append(torch.from_numpy(cc.adjacency_matrix(1, 2).todense()).to_sparse())

            B = cc.incidence_matrix(rank=1, to_rank=2)
            A = B.T @ B
            A.setdiag(0)
            coa2.append(torch.from_numpy(A.todense()).to_sparse())

            b1.append(torch.from_numpy(cc.incidence_matrix(0, 1).todense()).to_sparse())
            b2.append(torch.from_numpy(cc.incidence_matrix(1, 2).todense()).to_sparse())

        return a0, a1, coa2, b1, b2

    def num_classes(self):
        """Returns the number of classes in the dataset.

        Returns
        -------
        int
            Number of classes in the dataset.
        """
        return len(np.unique(self.y))

    def channels_dim(self):
        """Returns the number of channels for each input signal.

        Returns
        -------
        tuple of int
            Number of channels for each input signal.
        """
        return [self.x_0[0].shape[1], self.x_1[0].shape[1], self.x_2[0].shape[1]]

    def __len__(self):
        """Returns the number of elements in the dataset.

        Returns
        -------
        int
            Number of elements in the dataset.
        """
        return len(self.complexes)

    def __getitem__(self, idx):
        """Returns the idx-th element in the dataset.

        Parameters
        ----------
        idx : int
            Index of the element to return.

        Returns
        -------
        tuple of torch.Tensor
            Tuple containing the idx-th element in the dataset, including the input signals on nodes, edges and faces, the neighborhood matrices and the label.
        """
        return (
            self.x_0[idx],
            self.x_1[idx],
            self.x_2[idx],
            self.a0[idx],
            self.a1[idx],
            self.coa2[idx],
            self.b1[idx],
            self.b2[idx],
            self.y[idx],
        )

We load the data.

In [3]:
shrec_training, shrec_testing = datasets.shrec_16()

downloading shrec 16 full dataset...

done!
Loading shrec 16 full dataset...

done!


Creating the train dataset and dataloader.

In [7]:
training_dataset = SHRECDataset(shrec_training)
training_dataloader = DataLoader(training_dataset, batch_size=1, shuffle=True)

Creating the train dataset and dataloader.

In [8]:
testing_dataset = SHRECDataset(shrec_testing)
testing_dataloader = DataLoader(testing_dataset, batch_size=1, shuffle=True)

# Create the Neural Network


The task is to classify the meshes into their corresponding classes. To address this, we employ the Higher Order Attention Network Model for Mesh Classification, as outlined in the article [Higher Order Attention Networks](https://www.researchgate.net/publication/361022512_Higher-Order_Attention_Networks). This model integrates a hierarchical and attention-based message passing scheme as per the article's descriptions. In addition, the model utilizes a final sum pooling layer which effectively maps the nodal, edge, and face features of the meshes into a shared N-dimensional Euclidean space, where N represents the number of different classes.

In [13]:
import torch.nn as nn

class LTN_Classification(nn.Module): 
    def __init__(self, in_ch_v, in_ch_e, in_ch_f, out_dim, num_classes, activation=None,device=None):
        super(LTN_Classification, self).__init__()

        
        self.conv_v2v = Conv(in_channels=in_ch_v, out_channels=out_dim)
        self.conv_e2e = Conv(in_channels=in_ch_e, out_channels=out_dim)
        self.conv_f2f = Conv(in_channels=in_ch_f, out_channels=out_dim)

        self.conv_v2e = Conv(in_channels=out_dim, out_channels=out_dim)
        self.con_e2f = Conv(in_channels=out_dim, out_channels=out_dim)


        self.lin = nn.Linear(2 * out_dim, num_classes) 
        # self.lin2 = nn.Linear(out_dim, num_classes) 
        # self.lin3 = nn.Linear(out_dim, num_classes) 

        self.act = activation


    def forward(self, xv, xe, xf,
                gv2v, ge2e, gf2f, gv2e, ge2f ):
        zv1 = self.conv_v2v(xv, gv2v)
        ze1 = self.conv_e2e(xe, ge2e)
        zf1 = self.conv_f2f(xf, gf2f)
        
        zv1 = self.act(zv1)
        ze1 = self.act(ze1)
        zf1 = self.act(zf1)

        ze2 = self.conv_v2e(zv1, gv2e, ze1)
        zf2 = self.con_e2f(ze1, ge2f, zf1)


        
        zf2 = torch.mean(zf2, dim=0)
        zf2 = self.act(zf2)
        ze2 = torch.mean(zf2, dim=0)
        ze2 = self.act(ze2)

        print(f'zf2: {zf2.shape} ze2: {ze2.shape}')
        out = torch.cat([zf2, ze2], dim=0)
        out = self.lin(out)

        
        return out

# Train the Neural Network

We create the trainer class. The model is trained using the Adam optimizer and the Cross Entropy Loss function.

In [14]:
class Trainer:
    """Trainer for the HOANMeshClassifier.

    Parameters
    ----------
    model : torch.nn.Module
        The model to train.
    training_dataloader : torch.utils.data.DataLoader
        The dataloader for the training set.
    testing_dataloader : torch.utils.data.DataLoader
        The dataloader for the testing set.
    learning_rate : float
        The learning rate for the Adam optimizer.
    device : torch.device
        The device to use for training.
    """

    def __init__(
        self, model, training_dataloader, testing_dataloader, learning_rate, device
    ):
        self.model = model.to(device)
        self.training_dataloader = training_dataloader
        self.testing_dataloader = testing_dataloader
        self.device = device
        self.crit = torch.nn.CrossEntropyLoss()
        self.opt = torch.optim.Adam(model.parameters(), lr=learning_rate)

    def _to_device(self, x):
        """Converts tensors to the correct type and moves them to the device.

        Parameters
        ----------
        x : List[torch.Tensor]
            List of tensors to convert.
        Returns
        -------
        List[torch.Tensor]
            List of converted tensors to float type and moved to the device.
        """

        return [el[0].float().to(self.device) for el in x]

    def train(self, num_epochs=500, test_interval=25):
        """Trains the model for the specified number of epochs.

        Parameters
        ----------
        num_epochs : int
            Number of epochs to train.
        test_interval : int
            Interval between testing epochs.
        """
        for epoch_i in range(1, num_epochs + 1):
            training_accuracy, epoch_loss = self._train_epoch()
            print(
                f"Epoch: {epoch_i} loss: {epoch_loss:.4f} Train_acc: {training_accuracy:.4f}",
                flush=True,
            )
            if epoch_i % test_interval == 0:
                test_accuracy = self.validate()
                print(f"Test_acc: {test_accuracy:.4f}", flush=True)

    def _train_epoch(self):
        """Trains the model for one epoch.

        Returns
        -------
        training_accuracy : float
            The mean training accuracy for the epoch.
        epoch_loss : float
            The mean loss for the epoch.
        """
        training_samples = len(self.training_dataloader.dataset)
        total_loss = 0
        correct = 0
        self.model.train()
        for sample in self.training_dataloader:
            (
                x_0,
                x_1,
                x_2,
                adjacency_0,
                adjacency_1,
                coadjacency_2,
                incidence_1,
                incidence_2,
            ) = self._to_device(sample[:-1])

            self.opt.zero_grad()

            y_hat = self.model.forward(
                x_0,
                x_1,
                x_2,
                adjacency_0,
                adjacency_1,
                coadjacency_2,
                incidence_1.T,
                incidence_2.T,
            )

            y = sample[-1][0].long().to(self.device)
            total_loss += self._compute_loss_and_update(y_hat, y)
            correct += (y_hat.argmax() == y).sum().item()

        training_accuracy = correct / training_samples
        epoch_loss = total_loss / training_samples

        return training_accuracy, epoch_loss

    def _compute_loss_and_update(self, y_hat, y):
        """Computes the loss, performs backpropagation, and updates the model's parameters.

        Parameters
        ----------
        y_hat : torch.Tensor
            The output of the model.
        y : torch.Tensor
            The ground truth.

        Returns
        -------
        loss: float
            The loss value.
        """

        loss = self.crit(y_hat, y)
        loss.backward()
        self.opt.step()
        return loss.item()

    def validate(self):
        """Validates the model using the testing dataloader.

        Returns
        -------
        test_accuracy : float
            The mean testing accuracy.
        """
        correct = 0
        self.model.eval()
        test_samples = len(self.testing_dataloader.dataset)
        with torch.no_grad():
            for sample in self.testing_dataloader:
                (
                    x_0,
                    x_1,
                    x_2,
                    adjacency_0,
                    adjacency_1,
                    coadjacency_2,
                    incidence_1,
                    incidence_2,
                ) = self._to_device(sample[:-1])

                y_hat = self.model(
                    x_0,
                    x_1,
                    x_2,
                    adjacency_0,
                    adjacency_1,
                    coadjacency_2,
                    incidence_1.T,
                    incidence_2.T,
                )
                y = sample[-1][0].long().to(self.device)
                correct += (y_hat.argmax() == y).sum().item()
            test_accuracy = correct / test_samples
            return test_accuracy

We define the parameters for the model. We use softmax activation for the attention layers. Moreover, we use relu activation for the update and the aggregation steps. We set the negative slope parameter for the Leaky ReLU activation to 0.2. We only use one higher order attention layer as it already achieves almost perfect test accuracy, although more layers could be added.

In [15]:
in_channels = training_dataset.channels_dim()
intermediate_channels = [60, 60, 60]
final_channels = [60, 60, 60]

channels_per_layer = [[in_channels, intermediate_channels, final_channels]]
# defube HOAN mesh classifier
# model = Cxn_att_classification(
#     channels_per_layer, negative_slope=0.2, num_classes=training_dataset.num_classes()
# )
model = LTN_Classification(in_ch_v=in_channels[0],in_ch_e=in_channels[1], in_ch_f=in_channels[2], out_dim=60, num_classes=training_dataset.num_classes(), activation=nn.LeakyReLU(negative_slope=0.2))
# If GPU's are available, we will make use of them. Otherwise, this will run on CPU.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on {device}")
trainer = Trainer(model, training_dataloader, testing_dataloader, 0.001, device)

Training on cuda


We train the HoanMeshClassifier using low amount of epochs: we keep training minimal for the purpose of rapid testing.

In [16]:
trainer.train(num_epochs=2)

RuntimeError: zero-dimensional tensor (at position 1) cannot be concatenated

Letting the model train for longer, we can see that the model achieves an outstanding performance on both the training and testing sets.

In [ ]:
trainer.train(num_epochs=100, test_interval=10)

Epoch: 1 loss: 3.4085 Train_acc: 0.0271
Epoch: 2 loss: 3.4083 Train_acc: 0.0229
Epoch: 3 loss: 3.4081 Train_acc: 0.0229
Epoch: 4 loss: 3.4083 Train_acc: 0.0146
Epoch: 5 loss: 3.4062 Train_acc: 0.0187
Epoch: 6 loss: 3.4062 Train_acc: 0.0271
Epoch: 7 loss: 3.3952 Train_acc: 0.0312
Epoch: 8 loss: 3.2418 Train_acc: 0.0604
Epoch: 9 loss: 3.1035 Train_acc: 0.0813
Epoch: 10 loss: 3.0070 Train_acc: 0.0771
Test_acc: 0.0833
Epoch: 11 loss: 2.9556 Train_acc: 0.0938
Epoch: 12 loss: 2.9448 Train_acc: 0.0958
Epoch: 13 loss: 2.8683 Train_acc: 0.1271
Epoch: 14 loss: 2.8592 Train_acc: 0.1208
Epoch: 15 loss: 2.8816 Train_acc: 0.1417
Epoch: 16 loss: 2.7917 Train_acc: 0.1417
Epoch: 17 loss: 2.7517 Train_acc: 0.1729
Epoch: 18 loss: 2.7197 Train_acc: 0.1375
Epoch: 19 loss: 2.6759 Train_acc: 0.1667
Epoch: 20 loss: 2.5774 Train_acc: 0.1667
Test_acc: 0.1417
Epoch: 21 loss: 2.4718 Train_acc: 0.2208
Epoch: 22 loss: 2.4021 Train_acc: 0.2313
Epoch: 23 loss: 2.3240 Train_acc: 0.2708
Epoch: 24 loss: 2.3141 Train_acc